<a href="https://colab.research.google.com/github/swen90006-2024/Tutorial/blob/main/SWEN90006_Tutorial_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SWEN90006 Tutorial 4

## Introduction
The purpose of this tutorial is for you to gain a deeper understanding
of control- and data-flow techniques, and to compare these to input
partitioning techniques.

### Prepare the Java Kernel
Since Java is not natively supported by Colab, we need to run the following code to enable Java kernel on Colab.

1. Run the cell bellow (click it and press Shift+Enter),
2. Change the kernel to java_use (Runtime -> Change Runtime Type -> select **"java_use"** -> Save)
3. Try and run the following cells. The java kernel is ready if the you can load the JUnit library in the following cell. 

### Trouble shooting
  * There are two Java runtimes having similar names that you can select in step 2. If you accidentally select **java** instead of **java_use**, Colab will enter an indefinite loop, show **connecting**, and the bottom bar has a message saying "Connecting to **java_tcp** Google Compute Engine backend", you should delete the runtime (Runtime -> Disconnect and Delete Runtime), and run step 1 again. 
    * The working runtime has a log message saying it is **connecting to java Google Compute Enginge Backend**

In [ ]:
%%sh
# Install java kernel
wget -q https://github.com/SpencerPark/IJava/releases/download/v1.3.0/ijava-1.3.0.zip 
unzip -q ijava-1.3.0.zip 
python install.py

# Install proxy for the java kernel
wget -qO- https://gist.github.com/wenta0g/67289a9b2e54b8128109abb3aff2194b/archive/0707f5d61d156ce2830505679994b0f1a606589b.tar.gz | tar xvz --strip-components=1
python install_ipc_proxy_kernel.py --kernel=java --implementation=ipc_proxy_kernel.py

In [1]:
%%loadFromPOM

<dependency>
  <groupId>junit</groupId>
  <artifactId>junit</artifactId>
  <version>4.13.2</version>
</dependency>

# Part 1 Control Flow Criteria

White-box testing provides more information to the tester than black box testing. One special case is when the requirements is not correctly stated to perform Equivalence Partitioning, or a simple input leads to very complex system behaviour where a short requirements is not sufficient to describe all possible expected outcomes. 

## Working With the Program - Control flow analysis

In the first part of the tutorial, we provided a code snippet simulating image transformations done in the LibPNG library. 

LibPNG supports a list of image transformations when reading and writing a given image, for example, converting rgb images to gray or creating or deleting color channels. You do not need to know what these transformation means. Suppose in the LibPNG library, a 8-bit integer called `transformations` is used to represent the transformation to be performed. Four transformation will be performed. The condition to perform each condition is to check whether some part of the transformation flag is following certain patterns. For example, to perform transformation "second", the bit at index 1 and 2 must be set to 1. 

The definition of the "nth bit" is as follows, we start counting from the Least Significant Bit:

`7th bit -> 00000010 <- 0th bit` ; in this example, we say the **1st bit is set to 1**, the function `isSet(n)` is only *true* when `n = 1`. 

For simplisity, there's no bit that contributes to multiple transformations. 

In [2]:
import java.util.ArrayList;
import java.util.List;

public class MyClass {
    // Fields
    private int myField;
    private List<String> transformations = new ArrayList<>();
    
    // Constructor
    public MyClass(String binary) {
        this.myField = parseBinaryString(binary);
    }

    // reject if the binary string is not 8 bits, or it contains non-binary characters
    // parse the binary string to an integer
    public int parseBinaryString(String binary){

        if (binary.length() != 8 || !binary.matches("[01]+")) {
            //System.out.println("Invalid binary string");
            throw new IllegalArgumentException("must be a string of 8 chars, each char is either 0 or 1");
        }

        int decimal = Integer.parseInt(binary, 2);
        return decimal;

    }
    // Check if the n-th bit is 1
    public boolean isSet(int n) {
        return (myField & (1 << n)) != 0;
    }

    // Print the list of transformation to be performed
    public void PNG_initialize_transformation() {
        // Add more transformations as needed

        // The first transformation is set when the 0th bit is set
        if (isSet(0)) {
            transformations.add("First");
        }
        // The second transformation is set when both the 1st and 2nd bits are set 
        if (isSet(1) && isSet(2)) {
            transformations.add("Second");
        }

        // The third transformation is set when the 3rd bit is set and either the 4th or 5th bit is set
        if (isSet(3) && (isSet(4) || isSet(5))) {
            transformations.add("Third");
        }

        // The fourth transformation is set when the 6th bit is not set or the 7th bit is set
        if (!isSet(6) || isSet(7)) {
            transformations.add("Fourth");
        }
    }
    // Print out the list of transformations to be performed
    public void printTransformations() {
        System.out.println("Transformations to perform: " + transformations);
    }
}


In [6]:
// Please provide a string having at most 8 characters, and only 0s and 1s
// Other input will be rejected
String binary = "00010110"; 

MyClass myclass = new MyClass(binary);

// check which transformations to perform due to some combinations of flags
myclass.PNG_initialize_transformation();

// print out the transformations to be performed
myclass.printTransformations();


Transformations to perform: [Second, Fourth]


## Your Task

We will work on the different coverage criteria, looking at the `PNG_initialize_transformation` function. 


### Task 1:
Draw the control-flow graph for the `PNG_initialize_transformation` function. You may break the function up into basic blocks to simplify your CFG.

Recall that a basic block is a continuous sequence of statements where control flows from one statement to the next, a single point of entry, a single point of exit and no branches or loops.

### Task 2: 

 For the given set of test cases, calculate the `branch coverage` score, using the given template (You only need to fill in blanks in the table and the coverage score). If your set of test cases does not achieve 100% `branch coverage`, add some test cases that achieves `branch coverage`. 

**Hint:** is making a branch `true` equivalent to performing the corresponding transformation?  


Set of test cases:
- `binary = "00000111"`
- `binary = "10101010"`
- `binary = "11110000"`

| Branch          | Branch 1 | Branch 1  | Branch 2 | Branch 2 | Branch 3 | Branch 3 | Branch 4 | Branch 4 |
|-----------------|----------|-----------|----------|----------|----------|----------|----------|----------|
| Test objectives | True     | False     | True     | False    | True     | False    | True     | False    |
| 00000111        |        |           |          |          |          |          |          |          |
| 10101010        |          |           |          |         |         |          |         |          |
| 11110000        |          |           |          |         |          |         |         |          |
 
$coverage\_score = \frac{objectives\_achieved}{total\_objectives} $


### Task 3:

Design a set of test cases that achieves `condition coverage`. 

Note: we do not consider code optimizations during compilations. The implication is that for a branch like `if (a and b)`, an optimized code may not check `b` given `a` is false. But we will **always check both `a` and `b`**. 


### Task 4:

Design a set of test cases that achieves `decision/condition coverage`.
